# OpenAI Company Profile Tool
Get information from various sources
- Wikipedia
- Perplexity search
- Annual report item 1.

and generate a summary company profile using an OpenAI prompt. 


In [19]:
import os
import dotenv
import warnings

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import requests

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

# turn off excessive warnings
# warnings.filterwarnings('ignore')


True

In [2]:
print(f'pandas         {pd.__version__}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
openai         1.28.0
langchain      0.1.20
wikipedia      (1, 4, 0)


# Get info from Wikipedia

In [3]:
symbol = "MRK"
company = "Merck"

In [4]:
# langchain tool
wikipedia_run = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
results = wikipedia_run.run(company)
print(results)


/opt/anaconda3/envs/bb_agent/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/anaconda3/envs/bb_agent/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Merck & Co.
Summary: Merck & Co., Inc. is an American multinational pharmaceutical company headquartered in Rahway, New Jersey, and is named for Merck Group, founded in Germany in 1668, of which it was once the American arm. The company does business as Merck Sharp & Dohme or MSD outside the United States and Canada. It is one of the largest pharmaceutical companies in the world, generally ranking in the global top five by revenue.
Merck & Co. was originally established as the American affiliate of Merck Group in 1891. Merck develops and produces medicines, vaccines, biologic therapies and animal health products. It has multiple blockbuster drugs or products each with 2020 revenues including cancer immunotherapy, anti-diabetic medication and vaccines against HPV and chickenpox.
The company is ranked 71st on the 2022 Fortune 500 and 87th on the 2022 Forbes Global 2000, both based on 2021 revenues. In 2023, the company’s seat in Forbes Global 2000 was 73.

Page: Merck Group
Summary

In [5]:
# use wikipedia module to search directly

result = wikipedia.search(company)
print(result)


['Merck', 'Merck & Co.', 'Merck Group', 'Merck Manual of Diagnosis and Therapy', 'Merck Millipore', 'Merck family', 'Merck Records', 'Merck Serono', 'Friedrich Jacob Merck', 'Albert W. Merck']


In [6]:
search_term = 'Merck & Co.',


In [7]:
page_object = wikipedia.page(title=search_term, auto_suggest=False)

# printing title
print(page_object.original_title)

# printing links on that page object
#print(page_object.links[0:10])

# printing html of page_object
display(HTML(page_object.html()))


('Merck & Co.',)


In [8]:
print(page_object.summary)

Merck & Co., Inc. is an American multinational pharmaceutical company headquartered in Rahway, New Jersey, and is named for Merck Group, founded in Germany in 1668, of which it was once the American arm. The company does business as Merck Sharp & Dohme or MSD outside the United States and Canada. It is one of the largest pharmaceutical companies in the world, generally ranking in the global top five by revenue.
Merck & Co. was originally established as the American affiliate of Merck Group in 1891. Merck develops and produces medicines, vaccines, biologic therapies and animal health products. It has multiple blockbuster drugs or products each with 2020 revenues including cancer immunotherapy, anti-diabetic medication and vaccines against HPV and chickenpox.
The company is ranked 71st on the 2022 Fortune 500 and 87th on the 2022 Forbes Global 2000, both based on 2021 revenues. In 2023, the company’s seat in Forbes Global 2000 was 73.


In [9]:
# markdown source
wiki_page_content = page_object.content
print(wiki_page_content)

Merck & Co., Inc. is an American multinational pharmaceutical company headquartered in Rahway, New Jersey, and is named for Merck Group, founded in Germany in 1668, of which it was once the American arm. The company does business as Merck Sharp & Dohme or MSD outside the United States and Canada. It is one of the largest pharmaceutical companies in the world, generally ranking in the global top five by revenue.
Merck & Co. was originally established as the American affiliate of Merck Group in 1891. Merck develops and produces medicines, vaccines, biologic therapies and animal health products. It has multiple blockbuster drugs or products each with 2020 revenues including cancer immunotherapy, anti-diabetic medication and vaccines against HPV and chickenpox.
The company is ranked 71st on the 2022 Fortune 500 and 87th on the 2022 Forbes Global 2000, both based on 2021 revenues. In 2023, the company’s seat in Forbes Global 2000 was 73. 


== Products ==

The company develops medicines, va

# Get a Web profile using Perplexity


In [10]:
# get info combining perplexity search, wikipedia
# perplexity prompt

# company = "Medical Properties Trust"
# symbol = "MPW"

px_system_prompt = """
You will act as a securities analyst and investment advisor with deep knowledge of financial markets,
securities analysis, portfolio management. You will maintain a professional yet engaging tone,
in the style of a senior investment bank research analyst."
"""

px_user_prompt = f"""
You are an experienced securities analyst, with deep knowledge of fundamental
stock analysis. You will focus on {company} ({symbol}), and provide a
comprehensive analysis, in the style of a senior research analyst at a major
investment bank such as Goldman Sachs.

You will cover the following detailed topics:

Company Profile: An overview of {company}, including its lines of business,
historical background, and notable recent developments.

Major News: Significant recent events related to {company} or its industry
impacting its competitive position or stock performance, including major
economic and industry developments affecting supply and demand for its products,
significant regulatory and legal activty.

Financial Performance: Analysis of recent {company} financial results and
stock performance compared to expectations, changes in dividends and stock buybacks.

Analyst Coverage: Overview of recent changes to analysts' ratings, including
specific upgrades, downgrades, noting the firm and analyst. Discuss in detail
any recent reports from short sellers, including the analyst and firm name, and
specific claims.

Product Announcements: Description of new product launches, strategic initiatives,
and corporate restructurings.

Strategic Moves: Information on deals, partnerships, mergers, acquisitions, divestitures,
joint ventures, and major new business and revenue.

Capital Market Activities: Updates on any stock or bond issuances, special dividends,
and stock splits.

Management Changes: Summary of sgnificant changes in executive management roles
within {company}.

Stock Price Movements: Notable stock price changes and their driving factors.
"""

print(px_user_prompt)


You are an experienced securities analyst, with deep knowledge of fundamental
stock analysis. You will focus on Merck (MRK), and provide a
comprehensive analysis, in the style of a senior research analyst at a major
investment bank such as Goldman Sachs.

You will cover the following detailed topics:

Company Profile: An overview of Merck, including its lines of business,
historical background, and notable recent developments.

Major News: Significant recent events related to Merck or its industry
impacting its competitive position or stock performance, including major
economic and industry developments affecting supply and demand for its products,
significant regulatory and legal activty.

Financial Performance: Analysis of recent Merck financial results and
stock performance compared to expectations, changes in dividends and stock buybacks.

Analyst Coverage: Overview of recent changes to analysts' ratings, including
specific upgrades, downgrades, noting the firm and analyst. Discus

In [11]:
def get_perplexity_search_response(px_system_prompt, px_user_prompt):

    perplexity_url = "https://api.perplexity.ai/chat/completions"

    payload = {
        "model": "pplx-7b-online",
        "messages": [
            {
                "role": "system",
                "content": px_system_prompt,
            },
            {
                "role": "user",
                "content": px_user_prompt,
            }
        ]
    }

    perplexity_headers = {
        "Authorization": f"Bearer {os.getenv('PERPLEXITY_API_KEY')}",
        "accept": "application/json",
        "content-type": "application/json"
    }

    response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
    px_response_str = response.json()['choices'][0]['message']['content']
    # clean up stuff that might be interpreted by Markdown as latex
    px_response_str = px_response_str.replace("$", "\\\$")
    return px_response_str

px_response_str = get_perplexity_search_response(px_system_prompt, px_user_prompt)
display(Markdown(px_response_str))


**Company Profile:**
Merck & Co., Inc., commonly known as Merck, is an American multinational pharmaceutical company headquartered in Rahway, New Jersey. The company was founded in 1891 as the US subsidiary of Merck of Darmstadt, Germany, and was nationalized by the US government during World War I. Merck is a leading global healthcare company that discovers, develops, manufactures, and markets a wide range of pharmaceuticals, vaccines, and animal health products.

**Major News:**
Recent significant events related to Merck or its industry include:

* In January 2020, Merck acquired ArQule, a developer of ARQ 531, an oral Bruton's tyrosine kinase (BTK) inhibitor, for \\$2.7 billion.
* In June 2020, Merck acquired Themis Bioscience, a company focused on vaccines and immune-modulation therapies for infectious diseases, including COVID-19 and cancer.
* In September 2020, Merck acquired \\$1 billion of Seattle Genetics common stock and agreed to co-develop ladiratuzumab vedotin.
* In November 2020, Merck announced it would acquire VelosBio for \\$2.75 billion, developer of VLS-101, an antibody-drug conjugate designed to target Tyrosine kinase-like orphan receptor 1 (ROR1) in both hematological and solid tumors.
* In April 2021, Merck acquired Pandion Therapeutics for \\$1.85 billion, expanding its offering in treating autoimmune diseases.

**Financial Performance:**
Merck's financial performance has been mixed in recent years. The company's revenue has been impacted by the COVID-19 pandemic, with a decline in 2020. However, Merck has made significant progress in its pipeline, with several new product launches and acquisitions. The company's financial results have been affected by the pandemic, with a decline in revenue and earnings in 2020. However, Merck has maintained its dividend payout and has made significant progress in its pipeline.

**Analyst Coverage:**
Recent changes to analysts' ratings include:

* Berenberg analyst Luisa Hector raised the price target on Merck to \\$140.00 (from \\$130.00) while maintaining a Buy rating.
* Salim Syed of Mizuho Securities raised the price target on Merck to \\$138.00.

**Product Announcements:**
Merck has made several significant product announcements in recent years, including:

* The acquisition of ArQule, which developed ARQ 531, an oral Bruton's tyrosine kinase (BTK) inhibitor.
* The acquisition of Themis Bioscience, which focused on vaccines and immune-modulation therapies for infectious diseases, including COVID-19 and cancer.
* The acquisition of Pandion Therapeutics, which expanded Merck's offering in treating autoimmune diseases.

**Strategic Moves:**
Merck has made several strategic moves in recent years, including:

* The acquisition of ArQule, which expanded Merck's pipeline in oncology.
* The acquisition of Themis Bioscience, which expanded Merck's presence in vaccines and immune-modulation therapies.
* The acquisition of Pandion Therapeutics, which expanded Merck's offering in treating autoimmune diseases.

**Capital Market Activities:**
Merck has made several capital market activities in recent years, including:

* The issuance of debt securities in 2020.
* The repurchase of shares in 2020.

**Management Changes:**
There have been no significant changes in executive management roles within Merck in recent years.

**Stock Price Movements:**
Merck's stock price has been volatile in recent years, with a decline in 2020 due to the COVID-19 pandemic. However, the company's stock price has recovered in recent months, driven by progress in its pipeline and strong financial performance.

Overall, Merck is a leading global healthcare company with a strong pipeline and a long history of innovation. The company's financial performance has been impacted by the COVID-19 pandemic, but it has made significant progress in recent years and is well-positioned for future growth.

# Get item 1 from latest annual report

In [12]:
tools = {}
tools_dict = {}
tools_examples = {}

In [13]:
# use sec-api to parse the url, but it requires subscription

# def get_annual_report_company_profile(url):
#     """
#     Retrieves the company profile from item 1 of the 10-K annual report using sec_api module.

#     Args:
#         latest_10k_url (str): The URL of the latest 10-K report.

#     Returns:
#         str: The text content of the company profile section from the annual report.
#     """

#     extractorApi = ExtractorApi(os.getenv('SEC_API_KEY'))
#     item_1_text = extractorApi.get_section(url, "1", "text")
#     return item_1_text


# tools["get_annual_report_company_profile"] = {
#     "type": "function",
#     "function": {
#         "name": "get_annual_report_company_profile",
#         "description": "Retrieves the company profile from item 1 of the 10-K annual report based on a valid URL for the 10-K.",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "url": {
#                     "type": "string",
#                     "description": "The URL of the latest 10-K report."
#                 }
#             },
#             "required": ["url"]
#         }
#     }
# })
# tools_dict["get_annual_report_company_profile"] = get_annual_report_company_profile

# item_1_text = get_annual_report_company_profile(latest_10k_url)
# print(item_1_text)



# 10-K example
# url_10k = "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20231231.htm"

# item_1A_text = extractorApi.get_section(url_10k, "1A", "text")
# item_1_text = extractorApi.get_section(url_10k, "1", "text")

# item_7_html = extractorApi.get_section(url_10k, "7", "html")

# 10-Q example
# url_10q = "https://www.sec.gov/Archives/edgar/data/1318605/000095017022006034/tsla-20220331.htm"

# part2_item_1A_text = extractorApi.get_section(url_10q, "part2item1a", "text")

# 8-K example
# url_8k = "https://www.sec.gov/Archives/edgar/data/66600/000149315222016468/form8-k.htm"


In [14]:
# def get_latest_10k_url(symbol):
#     """
#     Retrieves the URL of the latest 10-K filing for a given symbol.

#     Args:
#         symbol (str): The symbol of the equity.

#     Returns:
#         str: The URL of the latest 10-K annual report filing, or None if not found.

#     """
#     retval = None
#     try:
#         obj = obb.equity.fundamental.filings(symbol, form_type='10-K')
#         r = obj.results[0]
#         retval = r.report_url
#     except Exception as exc:
#         print(exc)

#     return retval

# tools["get_latest_10k_url"] = {
#     "type": "function",
#     "function": {
#         "name": "get_latest_10k_url",
#         "description": "Retrieves the URL of the latest 10-K annual report filing for a given symbol.",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "symbol": {
#                     "type": "string",
#                     "description": "The symbol to get the 10-K URL for"
#                 }
#             },
#             "required": ["symbol"]
#         }
#     }
# })
# tools_dict["get_latest_10k_url"] = get_latest_10k_url

# latest_10k_url = get_latest_10k_url(symbol)
# latest_10k_url

In [15]:
# queryApi = QueryApi(api_key=os.environ['SEC_API_KEY'])

# query = {
#   "query": f"ticker:{symbol} AND filedAt:[2024-01-01 TO 2024-12-31] AND formType:\"10-K\"",
#   "from": "0",
#   "size": "100",
#   "sort": [{ "filedAt": { "order": "desc" } }]
# }

# response = queryApi.get_filings(query)
# response


In [21]:
def get_10k_item1_from_symbol(symbol):
    """
    Gets item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except exc as Exception:
        print(str(exc))
        return ""
    return item1_text

item_1_text = get_10k_item1_from_symbol(symbol)
len(item_1_text)


/opt/anaconda3/envs/bb_agent/lib/python3.11/site-packages/sec_parser/processing_engine/core.py:153: UserWarning: Invalid section type for part1item1a. Defaulting to InvalidTopSectionIn10Q.
  elements = step.process(elements)
/opt/anaconda3/envs/bb_agent/lib/python3.11/site-packages/sec_parser/processing_engine/core.py:153: UserWarning: Invalid section type for part2item7. Defaulting to InvalidTopSectionIn10Q.
  elements = step.process(elements)
/opt/anaconda3/envs/bb_agent/lib/python3.11/site-packages/sec_parser/processing_engine/core.py:153: UserWarning: Invalid section type for part2item7a. Defaulting to InvalidTopSectionIn10Q.
  elements = step.process(elements)
/opt/anaconda3/envs/bb_agent/lib/python3.11/site-packages/sec_parser/processing_engine/core.py:153: UserWarning: Invalid section type for part2item8. Defaulting to InvalidTopSectionIn10Q.
  elements = step.process(elements)
/opt/anaconda3/envs/bb_agent/lib/python3.11/site-packages/sec_parser/processing_engine/core.py:153: Us

190265

In [22]:
item_1_text[:1000]

'Business.\nMerck\xa0& Co., Inc. (Merck or the Company) is a global health care company that delivers innovative health solutions through its prescription medicines, including biologic therapies, vaccines and animal health products. The Company’s operations are principally managed on a product basis and include two operating segments, Pharmaceutical and Animal Health, both of which are reportable segments.The Pharmaceutical segment includes human health pharmaceutical and vaccine products. Human health pharmaceutical products consist of therapeutic and preventive agents, generally sold by prescription, for the treatment of human disorders. The Company sells these human health pharmaceutical products primarily to drug wholesalers and retailers, hospitals, government agencies and managed health care providers such as health maintenance organizations, pharmacy benefit managers and other institutions. Human health vaccine products consist of preventive pediatric, adolescent and adult vacci

# Combine info from Perplexity query, Wikipedia, annual report into a company profile

In [23]:
openai_system_prompt = """
You are an experienced securities analyst, with deep knowledge of fundamental
stock analysis. You will answer questions using the professional yet engaging style
of a senior research analyst at a major investment bank such as Goldman Sachs.
"""

openai_user_prompt = f"""You will provide a summary profile of {company} ({symbol}),
based on information from Wikipedia, a web search, and other information as provided
below. You will only use the information provided below. You will include all information
of interest to a stock investor.

Wikipedia profile:
{wiki_page_content}

Perplexity profile:
{px_response_str}

Annual report Item 1:
{item_1_text}
"""

print(openai_user_prompt)

You will provide a summary profile of Merck (MRK),
based on information from Wikipedia, a web search, and other information as provided
below. You will only use the information provided below. You will include all information
of interest to a stock investor.

Wikipedia profile:
Merck & Co., Inc. is an American multinational pharmaceutical company headquartered in Rahway, New Jersey, and is named for Merck Group, founded in Germany in 1668, of which it was once the American arm. The company does business as Merck Sharp & Dohme or MSD outside the United States and Canada. It is one of the largest pharmaceutical companies in the world, generally ranking in the global top five by revenue.
Merck & Co. was originally established as the American affiliate of Merck Group in 1891. Merck develops and produces medicines, vaccines, biologic therapies and animal health products. It has multiple blockbuster drugs or products each with 2020 revenues including cancer immunotherapy, anti-diabetic medic

In [24]:
MODEL = "gpt-4-turbo"

MAX_INPUT_TOKENS = 65536     # includes text of all headlines
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0

In [25]:
def count_tokens(text: str):
    # Get the appropriate encoding
    encoding = tiktoken.encoding_for_model(MODEL)

    # Encode the text into tokens
    tokens = encoding.encode(text)

    # Return the number of tokens
    return len(tokens)

# Example usage
token_count = count_tokens(openai_user_prompt)

print(token_count)


45869


In [26]:
def get_response(
    client,
    messages,
    verbose=False,
    model=MODEL,
    max_output_tokens=MAX_OUTPUT_TOKENS,
    max_retries=MAX_RETRIES,
    temperature=TEMPERATURE,
):
    """
    Calls the OpenAI client with messages and returns the response.

    Args:
        client (OpenAI.ChatCompletionClient): The OpenAI client used to make the API call.
        messages (str or list): The messages to send to the chat model. If a string is provided, it will be converted to a list with a single message.
        verbose (bool, optional): Whether to log the messages to the console. Defaults to False.
        model (str, optional): The model to use for chat completion. Defaults to MODEL.
        max_output_tokens (int, optional): The maximum number of tokens in the response. Defaults to MAX_OUTPUT_TOKENS.
        max_retries (int, optional): The maximum number of retries in case of API errors. Defaults to MAX_RETRIES.
        temperature (float, optional): The temperature parameter for generating diverse responses. Defaults to TEMPERATURE.

    Returns:
        dict: The response from the OpenAI API as a JSON object.

    Raises:
        Exception: If an error occurs during the API call.

    """
    if type(messages) is not list:  # allow passing one string for convenience
        messages = [{"role": "user", "content": messages}]

    if verbose:
        print("\n".join([str(msg) for msg in messages]))

    # truncate number of tokens
    # retry loop, have received untrapped 500 errors like too busy
    for i in range(max_retries):
        if i > 0:
            print(f"Attempt {i+1}...")
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_output_tokens,
            )
            # no exception thrown
            return response
        except Exception as error:
            print(f"An exception occurred on attempt {i+1}:", error)
            time.sleep(sleeptime)
            continue  # try again
        # retries exceeded if you got this far
    print("Retries exceeded.")
    return None


In [27]:
client = OpenAI()
response = get_response(client, openai_user_prompt)
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

display(Markdown(response_str))


**Company Overview:**
Merck & Co., Inc., known as MSD outside the United States and Canada, is a leading global pharmaceutical company headquartered in Rahway, New Jersey. It was established in 1891 as the American affiliate of the German company Merck Group. Merck develops, manufactures, and markets a wide range of pharmaceutical products, vaccines, biologic therapies, and animal health products.

**Key Products:**
Merck's portfolio includes several blockbuster drugs that generate significant revenue. These include Keytruda (cancer treatment), Januvia (type 2 diabetes treatment), and Gardasil (HPV vaccine). Keytruda, in particular, is a major revenue driver for the company, with sales significantly contributing to Merck's financial performance.

**Financial Performance:**
Merck is ranked among the top pharmaceutical companies globally, consistently appearing in the top five by revenue. It is listed on the Fortune 500 and Forbes Global 2000, indicating robust financial health and substantial revenue generation. Recent financial data shows a strong emphasis on research and development, aimed at sustaining the company's pipeline of innovative drugs and therapies.

**Market Position and Competition:**
Merck holds a strong market position, with its products serving a wide range of medical needs in oncology, diabetes, vaccines, and animal health. The company faces competition from both generic medications and other pharmaceutical giants. The loss of patent protection for key drugs poses a risk, potentially leading to significant declines in revenue as generic alternatives become available.

**Research and Development:**
Merck has a strong focus on R&D, investing heavily in the development of new pharmaceutical products and vaccines. This is critical for the company's growth, helping to offset revenues lost due to patent expirations of its leading drugs.

**Regulatory and Legal Landscape:**
The pharmaceutical industry is highly regulated. Merck must navigate complex regulatory environments across different countries, which can impact the approval and commercialization of its products. Additionally, the company faces legal challenges, including patent litigation and product liability claims, which could affect its financial status and market reputation.

**Strategic Initiatives:**
Merck actively pursues strategic initiatives, including mergers, acquisitions, and partnerships, to enhance its product portfolio and enter new markets. Recent acquisitions aim to strengthen its capabilities in cancer treatment and broaden its vaccine offerings.

**Challenges and Risks:**
Merck faces several challenges, including stringent regulatory approvals, competition from generics, patent expirations, and potential adverse effects from its products. Economic downturns and changes in healthcare policies can also impact Merck's profitability.

**Investor Considerations:**
For investors, Merck presents a profile of strong market presence, significant investment in R&D, and strategic initiatives aimed at long-term growth. However, risks related to patent expirations, competition, and regulatory challenges must be carefully considered. The company's commitment to innovation and its diverse product pipeline provide a positive outlook, though market and legal risks could impact future performance.

**Conclusion:**
Merck & Co., Inc. stands as a leader in the pharmaceutical industry, driven by a robust product portfolio and a strategic focus on research and development. While facing typical industry challenges, its proactive strategies in innovation and market expansion position it well for future growth. Investors should weigh its market strengths against the inherent risks in the pharmaceutical sector.